# Language Translation
In this project, you’re going to take a peek into the realm of neural network machine translation.  You’ll be training a sequence to sequence model on a dataset of English and French sentences that can translate new sentences from English to French.
## Get the Data
Since translating the whole language of English to French will take lots of time to train, we have provided you with a small portion of the English corpus.

In [1]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
import problem_unittests as tests

source_path = 'data/small_vocab_en'
target_path = 'data/small_vocab_fr'
source_text = helper.load_data(source_path)
target_text = helper.load_data(target_path)

## Explore the Data
Play around with view_sentence_range to view different parts of the data.

In [2]:
view_sentence_range = (0, 10)

"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in source_text.split()})))

sentences = source_text.split('\n')
word_counts = [len(sentence.split()) for sentence in sentences]
print('Number of sentences: {}'.format(len(sentences)))
print('Average number of words in a sentence: {}'.format(np.average(word_counts)))

print()
print('English sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(source_text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))
print()
print('French sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(target_text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 227
Number of sentences: 137861
Average number of words in a sentence: 13.225277634719028

English sentences 0 to 10:
new jersey is sometimes quiet during autumn , and it is snowy in april .
the united states is usually chilly during july , and it is usually freezing in november .
california is usually quiet during march , and it is usually hot in june .
the united states is sometimes mild during june , and it is cold in september .
your least liked fruit is the grape , but my least liked is the apple .
his favorite fruit is the orange , but my favorite is the grape .
paris is relaxing during december , but it is usually chilly in july .
new jersey is busy during spring , and it is never hot in march .
our least liked fruit is the lemon , but my least liked is the grape .
the united states is sometimes busy during january , and it is sometimes warm in november .

French sentences 0 to 10:
new jersey est parfois calme pendant l' automne 

## Implement Preprocessing Function
### Text to Word Ids
As you did with other RNNs, you must turn the text into a number so the computer can understand it. In the function `text_to_ids()`, you'll turn `source_text` and `target_text` from words to ids.  However, you need to add the `<EOS>` word id at the end of `target_text`.  This will help the neural network predict when the sentence should end.

You can get the `<EOS>` word id by doing:
```python
target_vocab_to_int['<EOS>']
```
You can get other word ids using `source_vocab_to_int` and `target_vocab_to_int`.

In [5]:
def text_to_ids(source_text, target_text, source_vocab_to_int, target_vocab_to_int):
    """
    Convert source and target text to proper word ids
    :param source_text: String that contains all the source text.
    :param target_text: String that contains all the target text.
    :param source_vocab_to_int: Dictionary to go from the source words to an id
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :return: A tuple of lists (source_id_text, target_id_text)
    """
    # TODO: Implement Function
    source_vocab_ids = [[source_vocab_to_int.get(vocab, source_vocab_to_int['<UNK>']) for vocab in line.split(' ')] for line in source_text.split('\n')]
    target_vocab_ids = [[target_vocab_to_int.get(vocab, target_vocab_to_int['<UNK>']) for vocab in line.split(' ')] + [target_vocab_to_int['<EOS>']] for line in target_text.split('\n')] 
    return source_vocab_ids, target_vocab_ids

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_text_to_ids(text_to_ids)

Tests Passed


### Preprocess all the data and save it
Running the code cell below will preprocess all the data and save it to file.

In [6]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
helper.preprocess_and_save_data(source_path, target_path, text_to_ids)

# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [7]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np
import helper
import problem_unittests as tests

(source_int_text, target_int_text), (source_vocab_to_int, target_vocab_to_int), _ = helper.load_preprocess()

### Check the Version of TensorFlow and Access to GPU
This will check to make sure you have the correct version of TensorFlow and access to a GPU

In [8]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from distutils.version import LooseVersion
import warnings
import tensorflow as tf
from tensorflow.python.layers.core import Dense

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.1'), 'Please use TensorFlow version 1.1 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.4.0
Default GPU Device: /device:GPU:0


## Build the Neural Network
You'll build the components necessary to build a Sequence-to-Sequence model by implementing the following functions below:
- `model_inputs`
- `process_decoder_input`
- `encoding_layer`
- `decoding_layer_train`
- `decoding_layer_infer`
- `decoding_layer`
- `seq2seq_model`

### Input
Implement the `model_inputs()` function to create TF Placeholders for the Neural Network. It should create the following placeholders:

- Input text placeholder named "input" using the TF Placeholder name parameter with rank 2.
- Targets placeholder with rank 2.
- Learning rate placeholder with rank 0.
- Keep probability placeholder named "keep_prob" using the TF Placeholder name parameter with rank 0.
- Target sequence length placeholder named "target_sequence_length" with rank 1
- Max target sequence length tensor named "max_target_len" getting its value from applying tf.reduce_max on the target_sequence_length placeholder. Rank 0.
- Source sequence length placeholder named "source_sequence_length" with rank 1

Return the placeholders in the following the tuple (input, targets, learning rate, keep probability, target sequence length, max target sequence length, source sequence length)

In [9]:
def model_inputs():
    """
    Create TF Placeholders for input, targets, learning rate, and lengths of source and target sequences.
    :return: Tuple (input, targets, learning rate, keep probability, target sequence length,
    max target sequence length, source sequence length)
    """
    # TODO: Implement Function
    input_data = tf.placeholder(tf.int32, [None, None], name='input')
    targets = tf.placeholder(tf.int32, [None, None], name='targets')
    lr = tf.placeholder(tf.float32, name='learning_rate')
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')
    target_sequence_length = tf.placeholder(tf.int32, (None,), name='target_sequence_length')
    max_target_len = tf.reduce_max(target_sequence_length, name='max_target_len')
    source_sequence_length = tf.placeholder(tf.int32, (None,), name='source_sequence_length')
    
    return input_data, targets, lr, keep_prob, target_sequence_length, max_target_len, source_sequence_length


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_model_inputs(model_inputs)

Tests Passed


### Process Decoder Input
Implement `process_decoder_input` by removing the last word id from each batch in `target_data` and concat the GO ID to the begining of each batch.

In [10]:
def process_decoder_input(target_data, target_vocab_to_int, batch_size):
    """
    Preprocess target data for encoding
    :param target_data: Target Placehoder
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :param batch_size: Batch Size
    :return: Preprocessed target data
    """
    # TODO: Implement Function
    ending = tf.strided_slice(target_data, [0, 0], [batch_size, -1], [1, 1])
    dec_input = tf.concat([tf.fill([batch_size, 1], target_vocab_to_int['<GO>']), ending], 1)
    return dec_input

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_process_encoding_input(process_decoder_input)

Tests Passed


### Encoding
Implement `encoding_layer()` to create a Encoder RNN layer:
 * Embed the encoder input using [`tf.contrib.layers.embed_sequence`](https://www.tensorflow.org/api_docs/python/tf/contrib/layers/embed_sequence)
 * Construct a [stacked](https://github.com/tensorflow/tensorflow/blob/6947f65a374ebf29e74bb71e36fd82760056d82c/tensorflow/docs_src/tutorials/recurrent.md#stacking-multiple-lstms) [`tf.contrib.rnn.LSTMCell`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/LSTMCell) wrapped in a [`tf.contrib.rnn.DropoutWrapper`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/DropoutWrapper)
 * Pass cell and embedded input to [`tf.nn.dynamic_rnn()`](https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn)

In [11]:
from imp import reload
reload(tests)

def encoding_layer(rnn_inputs, rnn_size, num_layers, keep_prob, 
                   source_sequence_length, source_vocab_size, 
                   encoding_embedding_size):
    """
    Create encoding layer
    :param rnn_inputs: Inputs for the RNN
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param keep_prob: Dropout keep probability
    :param source_sequence_length: a list of the lengths of each sequence in the batch
    :param source_vocab_size: vocabulary size of source data
    :param encoding_embedding_size: embedding size of source data
    :return: tuple (RNN output, RNN state)
    """
    # TODO: Implement Function
    # Encoder embedding
    enc_embed_input = tf.contrib.layers.embed_sequence(rnn_inputs, source_vocab_size, encoding_embedding_size)
    
    # RNN cell
    def make_cell(rnn_size):
        enc_cell = tf.contrib.rnn.LSTMCell(rnn_size, initializer=tf.random_uniform_initializer(-0.1, 0.1, seed=2))
        enc_cell = tf.contrib.rnn.DropoutWrapper(enc_cell, keep_prob)
        return enc_cell

    enc_cell = tf.contrib.rnn.MultiRNNCell([make_cell(rnn_size) for _ in range(num_layers)])
    
    rnn_output, rnn_state = tf.nn.dynamic_rnn(enc_cell, enc_embed_input, sequence_length=source_sequence_length, dtype=tf.float32)
    
    return rnn_output, rnn_state

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_encoding_layer(encoding_layer)

Tests Passed


### Decoding - Training
Create a training decoding layer:
* Create a [`tf.contrib.seq2seq.TrainingHelper`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/TrainingHelper) 
* Create a [`tf.contrib.seq2seq.BasicDecoder`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/BasicDecoder)
* Obtain the decoder outputs from [`tf.contrib.seq2seq.dynamic_decode`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/dynamic_decode)

In [12]:

def decoding_layer_train(encoder_state, dec_cell, dec_embed_input, 
                         target_sequence_length, max_summary_length, 
                         output_layer, keep_prob):
    """
    Create a decoding layer for training
    :param encoder_state: Encoder State
    :param dec_cell: Decoder RNN Cell
    :param dec_embed_input: Decoder embedded input
    :param target_sequence_length: The lengths of each sequence in the target batch
    :param max_summary_length: The length of the longest sequence in the batch
    :param output_layer: Function to apply the output layer
    :param keep_prob: Dropout keep probability
    :return: BasicDecoderOutput containing training logits and sample_id
    """
    # TODO: Implement Function
    dec_embed_input = tf.contrib.layers.dropout(dec_embed_input, keep_prob)
    
    training_helper = tf.contrib.seq2seq.TrainingHelper(inputs=dec_embed_input, sequence_length=target_sequence_length, 
                                                        time_major=False)
    
    training_decoder = tf.contrib.seq2seq.BasicDecoder(dec_cell, training_helper, encoder_state, output_layer)  

    raining_decoder_output = tf.contrib.seq2seq.dynamic_decode(training_decoder, impute_finished=True, 
                                                               maximum_iterations=max_summary_length)[0]
    
    
    return raining_decoder_output



"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer_train(decoding_layer_train)

Tests Passed


### Decoding - Inference
Create inference decoder:
* Create a [`tf.contrib.seq2seq.GreedyEmbeddingHelper`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/GreedyEmbeddingHelper)
* Create a [`tf.contrib.seq2seq.BasicDecoder`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/BasicDecoder)
* Obtain the decoder outputs from [`tf.contrib.seq2seq.dynamic_decode`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/dynamic_decode)

In [13]:
def decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, start_of_sequence_id,
                         end_of_sequence_id, max_target_sequence_length,
                         vocab_size, output_layer, batch_size, keep_prob):
    """
    Create a decoding layer for inference
    :param encoder_state: Encoder state
    :param dec_cell: Decoder RNN Cell
    :param dec_embeddings: Decoder embeddings
    :param start_of_sequence_id: GO ID
    :param end_of_sequence_id: EOS Id
    :param max_target_sequence_length: Maximum length of target sequences
    :param vocab_size: Size of decoder/target vocabulary
    :param decoding_scope: TenorFlow Variable Scope for decoding
    :param output_layer: Function to apply the output layer
    :param batch_size: Batch size
    :param keep_prob: Dropout keep probability
    :return: BasicDecoderOutput containing inference logits and sample_id
    """
    # TODO: Implement Function
    start_tokens = tf.tile(tf.constant([start_of_sequence_id], dtype=tf.int32), [batch_size], name='start_tokens')
    
    inference_helper = tf.contrib.seq2seq.GreedyEmbeddingHelper(dec_embeddings,
                                                                start_tokens,
                                                                end_of_sequence_id)
    inference_decoder = tf.contrib.seq2seq.BasicDecoder(dec_cell,
                                                        inference_helper,
                                                        encoder_state,
                                                        output_layer)
    inference_decoder_output = tf.contrib.seq2seq.dynamic_decode(inference_decoder,
                                                            impute_finished=True,
                                                            maximum_iterations=max_target_sequence_length)[0]
    return inference_decoder_output



"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer_infer(decoding_layer_infer)

Tests Passed


### Build the Decoding Layer
Implement `decoding_layer()` to create a Decoder RNN layer.

* Embed the target sequences
* Construct the decoder LSTM cell (just like you constructed the encoder cell above)
* Create an output layer to map the outputs of the decoder to the elements of our vocabulary
* Use the your `decoding_layer_train(encoder_state, dec_cell, dec_embed_input, target_sequence_length, max_target_sequence_length, output_layer, keep_prob)` function to get the training logits.
* Use your `decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, start_of_sequence_id, end_of_sequence_id, max_target_sequence_length, vocab_size, output_layer, batch_size, keep_prob)` function to get the inference logits.

Note: You'll need to use [tf.variable_scope](https://www.tensorflow.org/api_docs/python/tf/variable_scope) to share variables between training and inference.

In [14]:
def decoding_layer(dec_input, encoder_state,
                   target_sequence_length, max_target_sequence_length,
                   rnn_size,
                   num_layers, target_vocab_to_int, target_vocab_size,
                   batch_size, keep_prob, decoding_embedding_size):
    """
    Create decoding layer
    :param dec_input: Decoder input
    :param encoder_state: Encoder state
    :param target_sequence_length: The lengths of each sequence in the target batch
    :param max_target_sequence_length: Maximum length of target sequences
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :param target_vocab_size: Size of target vocabulary
    :param batch_size: The size of the batch
    :param keep_prob: Dropout keep probability
    :param decoding_embedding_size: Decoding embedding size
    :return: Tuple of (Training BasicDecoderOutput, Inference BasicDecoderOutput)
    """
    # TODO: Implement Function
    
    # Decoder embedding
    dec_embeddings = tf.Variable(tf.random_uniform([target_vocab_size, decoding_embedding_size]))
    dec_embed_input = tf.nn.embedding_lookup(dec_embeddings, dec_input)
    
    def make_cell(rnn_size):
        dec_cell = tf.contrib.rnn.LSTMCell(rnn_size, initializer=tf.random_uniform_initializer(-0.1, 0.1, seed=2))
        return dec_cell
    
    dec_cell = tf.contrib.rnn.MultiRNNCell([make_cell(rnn_size) for _ in range(num_layers)])
    
    output_layer = Dense(target_vocab_size,kernel_initializer = tf.truncated_normal_initializer(mean = 0.0, stddev=0.1))
  
    with tf.variable_scope("decode"):
        
        training_basic_decoder_output = decoding_layer_train(encoder_state, dec_cell, dec_embed_input, target_sequence_length, 
                                                         max_target_sequence_length, output_layer, keep_prob)
    
    with tf.variable_scope("decode", reuse=True):
        
        start_of_sequence_id = target_vocab_to_int['<GO>']
        end_of_sequence_id = target_vocab_to_int['<EOS>']

        inference_basic_decoder_output = decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, start_of_sequence_id,
                                                              end_of_sequence_id, max_target_sequence_length, 
                                                              target_vocab_size, output_layer, batch_size, keep_prob)
    return training_basic_decoder_output, inference_basic_decoder_output



"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer(decoding_layer)

Tests Passed


### Build the Neural Network
Apply the functions you implemented above to:

- Encode the input using your `encoding_layer(rnn_inputs, rnn_size, num_layers, keep_prob,  source_sequence_length, source_vocab_size, encoding_embedding_size)`.
- Process target data using your `process_decoder_input(target_data, target_vocab_to_int, batch_size)` function.
- Decode the encoded input using your `decoding_layer(dec_input, enc_state, target_sequence_length, max_target_sentence_length, rnn_size, num_layers, target_vocab_to_int, target_vocab_size, batch_size, keep_prob, dec_embedding_size)` function.

In [15]:
def seq2seq_model(input_data, target_data, keep_prob, batch_size,
                  source_sequence_length, target_sequence_length,
                  max_target_sentence_length,
                  source_vocab_size, target_vocab_size,
                  enc_embedding_size, dec_embedding_size,
                  rnn_size, num_layers, target_vocab_to_int):
    """
    Build the Sequence-to-Sequence part of the neural network
    :param input_data: Input placeholder
    :param target_data: Target placeholder
    :param keep_prob: Dropout keep probability placeholder
    :param batch_size: Batch Size
    :param source_sequence_length: Sequence Lengths of source sequences in the batch
    :param target_sequence_length: Sequence Lengths of target sequences in the batch
    :param source_vocab_size: Source vocabulary size
    :param target_vocab_size: Target vocabulary size
    :param enc_embedding_size: Decoder embedding size
    :param dec_embedding_size: Encoder embedding size
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :return: Tuple of (Training BasicDecoderOutput, Inference BasicDecoderOutput)
    """
    # TODO: Implement Function
    _, enc_state = encoding_layer(input_data, rnn_size, num_layers, keep_prob,  
                                  source_sequence_length, source_vocab_size, enc_embedding_size)
    dec_input = process_decoder_input(target_data, target_vocab_to_int, batch_size)
    training_basic_decoder_output, inference_basic_decoder_output = decoding_layer(dec_input, enc_state, 
                                                                                   target_sequence_length, 
                                                                                   max_target_sentence_length, 
                                                                                   rnn_size, num_layers, 
                                                                                   target_vocab_to_int, 
                                                                                   target_vocab_size, 
                                                                                   batch_size, keep_prob, 
                                                                                   dec_embedding_size)
    return training_basic_decoder_output, inference_basic_decoder_output


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_seq2seq_model(seq2seq_model)

Tests Passed


## Neural Network Training
### Hyperparameters
Tune the following parameters:

- Set `epochs` to the number of epochs.
- Set `batch_size` to the batch size.
- Set `rnn_size` to the size of the RNNs.
- Set `num_layers` to the number of layers.
- Set `encoding_embedding_size` to the size of the embedding for the encoder.
- Set `decoding_embedding_size` to the size of the embedding for the decoder.
- Set `learning_rate` to the learning rate.
- Set `keep_probability` to the Dropout keep probability
- Set `display_step` to state how many steps between each debug output statement

In [28]:
# Number of Epochs
epochs = 10
# Batch Size
batch_size = 128
# RNN Size
rnn_size = 50
# Number of Layers
num_layers = 2
# Embedding Size
encoding_embedding_size = 15
decoding_embedding_size = 15
# Learning Rate
learning_rate = 0.001
# Dropout Keep Probability
keep_probability = 0.75
display_step = 5

### Build the Graph
Build the graph using the neural network you implemented.

In [29]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
save_path = 'checkpoints/dev'
(source_int_text, target_int_text), (source_vocab_to_int, target_vocab_to_int), _ = helper.load_preprocess()
max_target_sentence_length = max([len(sentence) for sentence in source_int_text])

train_graph = tf.Graph()
with train_graph.as_default():
    input_data, targets, lr, keep_prob, target_sequence_length, max_target_sequence_length, source_sequence_length = model_inputs()

    #sequence_length = tf.placeholder_with_default(max_target_sentence_length, None, name='sequence_length')
    input_shape = tf.shape(input_data)

    train_logits, inference_logits = seq2seq_model(tf.reverse(input_data, [-1]),
                                                   targets,
                                                   keep_prob,
                                                   batch_size,
                                                   source_sequence_length,
                                                   target_sequence_length,
                                                   max_target_sequence_length,
                                                   len(source_vocab_to_int),
                                                   len(target_vocab_to_int),
                                                   encoding_embedding_size,
                                                   decoding_embedding_size,
                                                   rnn_size,
                                                   num_layers,
                                                   target_vocab_to_int)


    training_logits = tf.identity(train_logits.rnn_output, name='logits')
    inference_logits = tf.identity(inference_logits.sample_id, name='predictions')

    masks = tf.sequence_mask(target_sequence_length, max_target_sequence_length, dtype=tf.float32, name='masks')

    with tf.name_scope("optimization"):
        # Loss function
        cost = tf.contrib.seq2seq.sequence_loss(
            training_logits,
            targets,
            masks)

        # Optimizer
        optimizer = tf.train.AdamOptimizer(lr)

        # Gradient Clipping
        gradients = optimizer.compute_gradients(cost)
        capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
        train_op = optimizer.apply_gradients(capped_gradients)


Batch and pad the source and target sequences

In [30]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
def pad_sentence_batch(sentence_batch, pad_int):
    """Pad sentences with <PAD> so that each sentence of a batch has the same length"""
    max_sentence = max([len(sentence) for sentence in sentence_batch])
    return [sentence + [pad_int] * (max_sentence - len(sentence)) for sentence in sentence_batch]


def get_batches(sources, targets, batch_size, source_pad_int, target_pad_int):
    """Batch targets, sources, and the lengths of their sentences together"""
    for batch_i in range(0, len(sources)//batch_size):
        start_i = batch_i * batch_size

        # Slice the right amount for the batch
        sources_batch = sources[start_i:start_i + batch_size]
        targets_batch = targets[start_i:start_i + batch_size]

        # Pad
        pad_sources_batch = np.array(pad_sentence_batch(sources_batch, source_pad_int))
        pad_targets_batch = np.array(pad_sentence_batch(targets_batch, target_pad_int))

        # Need the lengths for the _lengths parameters
        pad_targets_lengths = []
        for target in pad_targets_batch:
            pad_targets_lengths.append(len(target))

        pad_source_lengths = []
        for source in pad_sources_batch:
            pad_source_lengths.append(len(source))

        yield pad_sources_batch, pad_targets_batch, pad_source_lengths, pad_targets_lengths


### Train
Train the neural network on the preprocessed data. If you have a hard time getting a good loss, check the forms to see if anyone is having the same problem.

In [31]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
def get_accuracy(target, logits):
    """
    Calculate accuracy
    """
    max_seq = max(target.shape[1], logits.shape[1])
    if max_seq - target.shape[1]:
        target = np.pad(
            target,
            [(0,0),(0,max_seq - target.shape[1])],
            'constant')
    if max_seq - logits.shape[1]:
        logits = np.pad(
            logits,
            [(0,0),(0,max_seq - logits.shape[1])],
            'constant')

    return np.mean(np.equal(target, logits))

# Split data to training and validation sets
train_source = source_int_text[batch_size:]
train_target = target_int_text[batch_size:]
valid_source = source_int_text[:batch_size]
valid_target = target_int_text[:batch_size]
(valid_sources_batch, valid_targets_batch, valid_sources_lengths, valid_targets_lengths ) = next(get_batches(valid_source,
                                                                                                             valid_target,
                                                                                                             batch_size,
                                                                                                             source_vocab_to_int['<PAD>'],
                                                                                                             target_vocab_to_int['<PAD>']))                                                                                                  
with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(epochs):
        for batch_i, (source_batch, target_batch, sources_lengths, targets_lengths) in enumerate(
                get_batches(train_source, train_target, batch_size,
                            source_vocab_to_int['<PAD>'],
                            target_vocab_to_int['<PAD>'])):

            _, loss = sess.run(
                [train_op, cost],
                {input_data: source_batch,
                 targets: target_batch,
                 lr: learning_rate,
                 target_sequence_length: targets_lengths,
                 source_sequence_length: sources_lengths,
                 keep_prob: keep_probability})


            if batch_i % display_step == 0 and batch_i > 0:


                batch_train_logits = sess.run(
                    inference_logits,
                    {input_data: source_batch,
                     source_sequence_length: sources_lengths,
                     target_sequence_length: targets_lengths,
                     keep_prob: 1.0})


                batch_valid_logits = sess.run(
                    inference_logits,
                    {input_data: valid_sources_batch,
                     source_sequence_length: valid_sources_lengths,
                     target_sequence_length: valid_targets_lengths,
                     keep_prob: 1.0})

                train_acc = get_accuracy(target_batch, batch_train_logits)

                valid_acc = get_accuracy(valid_targets_batch, batch_valid_logits)

                print('Epoch {:>3} Batch {:>4}/{} - Train Accuracy: {:>6.4f}, Validation Accuracy: {:>6.4f}, Loss: {:>6.4f}'
                      .format(epoch_i, batch_i, len(source_int_text) // batch_size, train_acc, valid_acc, loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_path)
    print('Model Trained and Saved')

Epoch   0 Batch    5/1077 - Train Accuracy: 0.2605, Validation Accuracy: 0.3050, Loss: 5.7456
Epoch   0 Batch   10/1077 - Train Accuracy: 0.1982, Validation Accuracy: 0.3050, Loss: 5.5051
Epoch   0 Batch   15/1077 - Train Accuracy: 0.2355, Validation Accuracy: 0.3050, Loss: 5.0194
Epoch   0 Batch   20/1077 - Train Accuracy: 0.2316, Validation Accuracy: 0.3050, Loss: 4.6227
Epoch   0 Batch   25/1077 - Train Accuracy: 0.2387, Validation Accuracy: 0.3054, Loss: 4.2730
Epoch   0 Batch   30/1077 - Train Accuracy: 0.2277, Validation Accuracy: 0.3050, Loss: 4.0188
Epoch   0 Batch   35/1077 - Train Accuracy: 0.2340, Validation Accuracy: 0.3050, Loss: 3.8373
Epoch   0 Batch   40/1077 - Train Accuracy: 0.2434, Validation Accuracy: 0.3185, Loss: 3.7243
Epoch   0 Batch   45/1077 - Train Accuracy: 0.2629, Validation Accuracy: 0.3349, Loss: 3.6260
Epoch   0 Batch   50/1077 - Train Accuracy: 0.2590, Validation Accuracy: 0.3356, Loss: 3.5945
Epoch   0 Batch   55/1077 - Train Accuracy: 0.2891, Validati

Epoch   0 Batch  445/1077 - Train Accuracy: 0.3988, Validation Accuracy: 0.4663, Loss: 2.5972
Epoch   0 Batch  450/1077 - Train Accuracy: 0.4121, Validation Accuracy: 0.4485, Loss: 2.4742
Epoch   0 Batch  455/1077 - Train Accuracy: 0.4595, Validation Accuracy: 0.4702, Loss: 2.2699
Epoch   0 Batch  460/1077 - Train Accuracy: 0.4078, Validation Accuracy: 0.4730, Loss: 2.4961
Epoch   0 Batch  465/1077 - Train Accuracy: 0.3902, Validation Accuracy: 0.4737, Loss: 2.6462
Epoch   0 Batch  470/1077 - Train Accuracy: 0.4083, Validation Accuracy: 0.4741, Loss: 2.5236
Epoch   0 Batch  475/1077 - Train Accuracy: 0.4398, Validation Accuracy: 0.4755, Loss: 2.3218
Epoch   0 Batch  480/1077 - Train Accuracy: 0.4243, Validation Accuracy: 0.4798, Loss: 2.4882
Epoch   0 Batch  485/1077 - Train Accuracy: 0.4266, Validation Accuracy: 0.4787, Loss: 2.4330
Epoch   0 Batch  490/1077 - Train Accuracy: 0.4090, Validation Accuracy: 0.4759, Loss: 2.4499
Epoch   0 Batch  495/1077 - Train Accuracy: 0.4316, Validati

Epoch   0 Batch  885/1077 - Train Accuracy: 0.5174, Validation Accuracy: 0.5181, Loss: 1.6410
Epoch   0 Batch  890/1077 - Train Accuracy: 0.5234, Validation Accuracy: 0.5199, Loss: 1.6484
Epoch   0 Batch  895/1077 - Train Accuracy: 0.4395, Validation Accuracy: 0.5217, Loss: 1.7961
Epoch   0 Batch  900/1077 - Train Accuracy: 0.4586, Validation Accuracy: 0.5220, Loss: 1.8718
Epoch   0 Batch  905/1077 - Train Accuracy: 0.4879, Validation Accuracy: 0.5256, Loss: 1.7744
Epoch   0 Batch  910/1077 - Train Accuracy: 0.4940, Validation Accuracy: 0.5206, Loss: 1.7157
Epoch   0 Batch  915/1077 - Train Accuracy: 0.4165, Validation Accuracy: 0.5273, Loss: 1.9357
Epoch   0 Batch  920/1077 - Train Accuracy: 0.4730, Validation Accuracy: 0.5241, Loss: 1.7777
Epoch   0 Batch  925/1077 - Train Accuracy: 0.4725, Validation Accuracy: 0.5291, Loss: 1.7134
Epoch   0 Batch  930/1077 - Train Accuracy: 0.4910, Validation Accuracy: 0.5270, Loss: 1.7218
Epoch   0 Batch  935/1077 - Train Accuracy: 0.4695, Validati

Epoch   1 Batch  250/1077 - Train Accuracy: 0.3477, Validation Accuracy: 0.3622, Loss: 1.1707
Epoch   1 Batch  255/1077 - Train Accuracy: 0.3258, Validation Accuracy: 0.3491, Loss: 1.2605
Epoch   1 Batch  260/1077 - Train Accuracy: 0.3263, Validation Accuracy: 0.3391, Loss: 1.1783
Epoch   1 Batch  265/1077 - Train Accuracy: 0.3758, Validation Accuracy: 0.3690, Loss: 1.2061
Epoch   1 Batch  270/1077 - Train Accuracy: 0.2895, Validation Accuracy: 0.3320, Loss: 1.3043
Epoch   1 Batch  275/1077 - Train Accuracy: 0.3731, Validation Accuracy: 0.3239, Loss: 1.1548
Epoch   1 Batch  280/1077 - Train Accuracy: 0.3297, Validation Accuracy: 0.3221, Loss: 1.2315
Epoch   1 Batch  285/1077 - Train Accuracy: 0.3839, Validation Accuracy: 0.3558, Loss: 1.1418
Epoch   1 Batch  290/1077 - Train Accuracy: 0.3801, Validation Accuracy: 0.4169, Loss: 1.2266
Epoch   1 Batch  295/1077 - Train Accuracy: 0.3544, Validation Accuracy: 0.4251, Loss: 1.2860
Epoch   1 Batch  300/1077 - Train Accuracy: 0.3520, Validati

Epoch   1 Batch  690/1077 - Train Accuracy: 0.5016, Validation Accuracy: 0.5202, Loss: 0.9826
Epoch   1 Batch  695/1077 - Train Accuracy: 0.4938, Validation Accuracy: 0.5327, Loss: 0.9614
Epoch   1 Batch  700/1077 - Train Accuracy: 0.4633, Validation Accuracy: 0.5320, Loss: 0.9600
Epoch   1 Batch  705/1077 - Train Accuracy: 0.4856, Validation Accuracy: 0.5252, Loss: 1.0433
Epoch   1 Batch  710/1077 - Train Accuracy: 0.4551, Validation Accuracy: 0.5348, Loss: 0.9766
Epoch   1 Batch  715/1077 - Train Accuracy: 0.4773, Validation Accuracy: 0.5320, Loss: 1.0159
Epoch   1 Batch  720/1077 - Train Accuracy: 0.4548, Validation Accuracy: 0.5146, Loss: 1.0515
Epoch   1 Batch  725/1077 - Train Accuracy: 0.5033, Validation Accuracy: 0.5387, Loss: 0.9311
Epoch   1 Batch  730/1077 - Train Accuracy: 0.5086, Validation Accuracy: 0.5270, Loss: 0.9639
Epoch   1 Batch  735/1077 - Train Accuracy: 0.5008, Validation Accuracy: 0.5249, Loss: 0.9902
Epoch   1 Batch  740/1077 - Train Accuracy: 0.4926, Validati

...

Epoch   8 Batch  205/1077 - Train Accuracy: 0.7273, Validation Accuracy: 0.7198, Loss: 0.3310
Epoch   8 Batch  210/1077 - Train Accuracy: 0.7500, Validation Accuracy: 0.7191, Loss: 0.3108
Epoch   8 Batch  215/1077 - Train Accuracy: 0.7152, Validation Accuracy: 0.7244, Loss: 0.3385
Epoch   8 Batch  220/1077 - Train Accuracy: 0.7385, Validation Accuracy: 0.7212, Loss: 0.3286
Epoch   8 Batch  225/1077 - Train Accuracy: 0.7793, Validation Accuracy: 0.7358, Loss: 0.3233
Epoch   8 Batch  230/1077 - Train Accuracy: 0.7500, Validation Accuracy: 0.7337, Loss: 0.3102
Epoch   8 Batch  235/1077 - Train Accuracy: 0.7719, Validation Accuracy: 0.7344, Loss: 0.3121
Epoch   8 Batch  240/1077 - Train Accuracy: 0.7762, Validation Accuracy: 0.7379, Loss: 0.3016
Epoch   8 Batch  245/1077 - Train Accuracy: 0.7775, Validation Accuracy: 0.7241, Loss: 0.3083
Epoch   8 Batch  250/1077 - Train Accuracy: 0.7305, Validation Accuracy: 0.7315, Loss: 0.3002
Epoch   8 Batch  255/1077 - Train Accuracy: 0.7156, Validati

Epoch   8 Batch  645/1077 - Train Accuracy: 0.7623, Validation Accuracy: 0.7351, Loss: 0.2998
Epoch   8 Batch  650/1077 - Train Accuracy: 0.7383, Validation Accuracy: 0.7188, Loss: 0.3200
Epoch   8 Batch  655/1077 - Train Accuracy: 0.7137, Validation Accuracy: 0.7276, Loss: 0.3130
Epoch   8 Batch  660/1077 - Train Accuracy: 0.7477, Validation Accuracy: 0.7198, Loss: 0.3234
Epoch   8 Batch  665/1077 - Train Accuracy: 0.7395, Validation Accuracy: 0.7283, Loss: 0.3189
Epoch   8 Batch  670/1077 - Train Accuracy: 0.7759, Validation Accuracy: 0.7330, Loss: 0.2885
Epoch   8 Batch  675/1077 - Train Accuracy: 0.7705, Validation Accuracy: 0.7216, Loss: 0.3111
Epoch   8 Batch  680/1077 - Train Accuracy: 0.7593, Validation Accuracy: 0.7450, Loss: 0.3162
Epoch   8 Batch  685/1077 - Train Accuracy: 0.7566, Validation Accuracy: 0.7326, Loss: 0.3211
Epoch   8 Batch  690/1077 - Train Accuracy: 0.7383, Validation Accuracy: 0.7333, Loss: 0.3146
Epoch   8 Batch  695/1077 - Train Accuracy: 0.7496, Validati

Epoch   9 Batch   10/1077 - Train Accuracy: 0.7751, Validation Accuracy: 0.7365, Loss: 0.3107
Epoch   9 Batch   15/1077 - Train Accuracy: 0.7801, Validation Accuracy: 0.7322, Loss: 0.2938
Epoch   9 Batch   20/1077 - Train Accuracy: 0.7766, Validation Accuracy: 0.7500, Loss: 0.2920
Epoch   9 Batch   25/1077 - Train Accuracy: 0.7457, Validation Accuracy: 0.7418, Loss: 0.2851
Epoch   9 Batch   30/1077 - Train Accuracy: 0.7711, Validation Accuracy: 0.7369, Loss: 0.2953
Epoch   9 Batch   35/1077 - Train Accuracy: 0.7852, Validation Accuracy: 0.7472, Loss: 0.2888
Epoch   9 Batch   40/1077 - Train Accuracy: 0.7617, Validation Accuracy: 0.7440, Loss: 0.3017
Epoch   9 Batch   45/1077 - Train Accuracy: 0.7484, Validation Accuracy: 0.7518, Loss: 0.3030
Epoch   9 Batch   50/1077 - Train Accuracy: 0.7605, Validation Accuracy: 0.7475, Loss: 0.2964
Epoch   9 Batch   55/1077 - Train Accuracy: 0.7477, Validation Accuracy: 0.7436, Loss: 0.2967
Epoch   9 Batch   60/1077 - Train Accuracy: 0.7667, Validati

Epoch   9 Batch  450/1077 - Train Accuracy: 0.7781, Validation Accuracy: 0.7390, Loss: 0.2831
Epoch   9 Batch  455/1077 - Train Accuracy: 0.7663, Validation Accuracy: 0.7418, Loss: 0.2649
Epoch   9 Batch  460/1077 - Train Accuracy: 0.7664, Validation Accuracy: 0.7536, Loss: 0.2995
Epoch   9 Batch  465/1077 - Train Accuracy: 0.7558, Validation Accuracy: 0.7496, Loss: 0.3182
Epoch   9 Batch  470/1077 - Train Accuracy: 0.7562, Validation Accuracy: 0.7571, Loss: 0.2950
Epoch   9 Batch  475/1077 - Train Accuracy: 0.7707, Validation Accuracy: 0.7539, Loss: 0.2753
Epoch   9 Batch  480/1077 - Train Accuracy: 0.8006, Validation Accuracy: 0.7532, Loss: 0.2842
Epoch   9 Batch  485/1077 - Train Accuracy: 0.7555, Validation Accuracy: 0.7543, Loss: 0.2924
Epoch   9 Batch  490/1077 - Train Accuracy: 0.7508, Validation Accuracy: 0.7589, Loss: 0.2952
Epoch   9 Batch  495/1077 - Train Accuracy: 0.7465, Validation Accuracy: 0.7422, Loss: 0.2736
Epoch   9 Batch  500/1077 - Train Accuracy: 0.7508, Validati

Epoch   9 Batch  890/1077 - Train Accuracy: 0.8036, Validation Accuracy: 0.7550, Loss: 0.2553
Epoch   9 Batch  895/1077 - Train Accuracy: 0.7645, Validation Accuracy: 0.7504, Loss: 0.2702
Epoch   9 Batch  900/1077 - Train Accuracy: 0.8250, Validation Accuracy: 0.7454, Loss: 0.2726
Epoch   9 Batch  905/1077 - Train Accuracy: 0.7891, Validation Accuracy: 0.7575, Loss: 0.2584
Epoch   9 Batch  910/1077 - Train Accuracy: 0.7946, Validation Accuracy: 0.7397, Loss: 0.2680
Epoch   9 Batch  915/1077 - Train Accuracy: 0.7377, Validation Accuracy: 0.7486, Loss: 0.2885
Epoch   9 Batch  920/1077 - Train Accuracy: 0.7660, Validation Accuracy: 0.7607, Loss: 0.2725
Epoch   9 Batch  925/1077 - Train Accuracy: 0.7846, Validation Accuracy: 0.7738, Loss: 0.2607
Epoch   9 Batch  930/1077 - Train Accuracy: 0.8066, Validation Accuracy: 0.7770, Loss: 0.2577
Epoch   9 Batch  935/1077 - Train Accuracy: 0.7824, Validation Accuracy: 0.7610, Loss: 0.2871
Epoch   9 Batch  940/1077 - Train Accuracy: 0.7793, Validati

### Save Parameters
Save the `batch_size` and `save_path` parameters for inference.

In [32]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Save parameters for checkpoint
helper.save_params(save_path)

# Checkpoint

In [33]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import tensorflow as tf
import numpy as np
import helper
import problem_unittests as tests

_, (source_vocab_to_int, target_vocab_to_int), (source_int_to_vocab, target_int_to_vocab) = helper.load_preprocess()
load_path = helper.load_params()

## Sentence to Sequence
To feed a sentence into the model for translation, you first need to preprocess it.  Implement the function `sentence_to_seq()` to preprocess new sentences.

- Convert the sentence to lowercase
- Convert words into ids using `vocab_to_int`
 - Convert words not in the vocabulary, to the `<UNK>` word id.

In [34]:
def sentence_to_seq(sentence, vocab_to_int):
    """
    Convert a sentence to a sequence of ids
    :param sentence: String
    :param vocab_to_int: Dictionary to go from the words to an id
    :return: List of word ids
    """
    # TODO: Implement Function
    sentence = sentence.lower()
    vocab_ids = [vocab_to_int.get(vocab, vocab_to_int['<UNK>']) 
                  for vocab in sentence.split(' ')]
    return vocab_ids


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_sentence_to_seq(sentence_to_seq)

Tests Passed


## Translate
This will translate `translate_sentence` from English to French.

In [35]:
translate_sentence = 'he saw a old yellow truck .'


"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
translate_sentence = sentence_to_seq(translate_sentence, source_vocab_to_int)

loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_path + '.meta')
    loader.restore(sess, load_path)

    input_data = loaded_graph.get_tensor_by_name('input:0')
    logits = loaded_graph.get_tensor_by_name('predictions:0')
    target_sequence_length = loaded_graph.get_tensor_by_name('target_sequence_length:0')
    source_sequence_length = loaded_graph.get_tensor_by_name('source_sequence_length:0')
    keep_prob = loaded_graph.get_tensor_by_name('keep_prob:0')

    translate_logits = sess.run(logits, {input_data: [translate_sentence]*batch_size,
                                         target_sequence_length: [len(translate_sentence)*2]*batch_size,
                                         source_sequence_length: [len(translate_sentence)]*batch_size,
                                         keep_prob: 1.0})[0]

print('Input')
print('  Word Ids:      {}'.format([i for i in translate_sentence]))
print('  English Words: {}'.format([source_int_to_vocab[i] for i in translate_sentence]))

print('\nPrediction')
print('  Word Ids:      {}'.format([i for i in translate_logits]))
print('  French Words: {}'.format(" ".join([target_int_to_vocab[i] for i in translate_logits])))


INFO:tensorflow:Restoring parameters from checkpoints/dev
Input
  Word Ids:      [215, 78, 94, 114, 183, 88, 106]
  English Words: ['he', 'saw', 'a', 'old', 'yellow', 'truck', '.']

Prediction
  Word Ids:      [287, 183, 320, 252, 190, 219, 1]
  French Words: il conduisait une voiture jaune . <EOS>


## Imperfect Translation
You might notice that some sentences translate better than others.  Since the dataset you're using only has a vocabulary of 227 English words of the thousands that you use, you're only going to see good results using these words.  For this project, you don't need a perfect translation. However, if you want to create a better translation model, you'll need better data.

You can train on the [WMT10 French-English corpus](http://www.statmt.org/wmt10/training-giga-fren.tar).  This dataset has more vocabulary and richer in topics discussed.  However, this will take you days to train, so make sure you've a GPU and the neural network is performing well on dataset we provided.  Just make sure you play with the WMT10 corpus after you've submitted this project.
## Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_language_translation.ipynb" and save it as a HTML file under "File" -> "Download as". Include the "helper.py" and "problem_unittests.py" files in your submission.